# Setup

In [1]:
# load the .env as environment variables
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import asyncio
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import BaseMessage
import pdfkit
import markdown
from PyPDF2 import PdfMerger
from tqdm.notebook import tqdm
import nest_asyncio

In [ ]:
nest_asyncio.apply()

# モデルクライアントの作成
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
)

point_prompts = (
        "- 常にTypeScriptの概念や実装をPythonで相当するものと対比して説明しているか。特に異なる点について必ず言及しているか\n"
        "- Pythonの形ヒントやdataclass, Pydanticなどを使って実装した場合のコードをTypeScriptのコードが出る度に記載しているか。TypeScriptのコードブロックとPythonのコードブロックの数は一致するようにしなければなりません。\n"
        "- 用語の違いや思想の違いなど含めて、Well-definedに記述されているか。\n"
        "-　抽象的な説明の場合でも、実践をイメージしやすいように常にコードとセットで説明しているか。可能な限り、TypeScriptとPython両方のコードを示す。\n"
        "-　内容は網羅的か。\n"
        "- 前のレビュー内容がある場合は、それが適切に反映されているか。"
    )
# 各エージェントの作成
writer_agent = AssistantAgent(
    name="WriterAgent",
    model_client=model_client,
    system_message="あなたはWriterAgentです。\n"
    "あなたはPythonとTypeScriptの両方に精通したソフトウェアエンジニアであり、「Python上級者のためのTypeScript」という本を執筆しています。\n"
    "執筆する際には以下の点に注意してください。\n"
    f"{point_prompts}\n"
    "- 前のレビュー内容がある場合は、それが適切に反映されているか。"
)

reviewer_agent = AssistantAgent(
    name="ReviewerAgent",
    model_client=model_client,
    system_message="あなたはReviewerAgentです。\n"
    "あなたはわかりやすく網羅的な技術書を執筆するエキスパートです。WriterAgentが作成したドラフトをレビューし、技術的な正確性と深さを確認の上、フィードバックを作成してください。\n"
    "また、そのクオリティが十分である場合は、APPROVEと返してください。\n"
    "もし、修正が必要な場合は、WriterAgentにフィードバックを送り、修正を依頼してください。"
    "レビューにあたっては以下の観点に注意してください。\n"
    f"{point_prompts}\n"
    "など。"
)

# editor_agent = AssistantAgent(
#     name="EditorAgent",
#     model_client=model_client,
#     system_message="あなたはEditorAgentです。レビュー済みのドラフトを編集し、文体や用語の統一を行ってください。"
# )

# 終了条件の設定
termination_condition = TextMentionTermination("APPROVE") # | MaxMessageTermination(max_messages=10)

selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

# エージェントチームの作成
# agent_team = SelectorGroupChat(
#     [writer_agent, reviewer_agent],
#     model_client=model_client,
#     termination_condition=termination_condition,
#     selector_prompt=selector_prompt,
#     # allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
# )

agent_team = RoundRobinGroupChat(
    [writer_agent, reviewer_agent],
    termination_condition=termination_condition,
)

# 各章のタイトルリスト
chapter_titles = [
    "01.TypeScriptの基礎",
    "02.TypeScriptの型システム",
    "03.インターフェースと型エイリアス",
    "04.型推論とユーティリティ型",
    "05.関数型プログラミングとTypeScript",
    "06.ジェネリクスと高度な型",
    "07.TypeScriptにおけるクラスとオブジェクト指向",
    "08.TypeScriptとPythonの設計思想の違い",
    "09.Pythonの型ヒントとTypeScriptの型の比較",
    "10.TypeScriptでの非同期処理",
    "11.型安全なAPI設計",
    "12.TypeScriptでのCLIツール開発",
    "13.フロントエンドとバックエンドの開発",
    "14.型安全なデータ処理",
    "15.TypeScriptの型システムの内部構造",
    "16.TypeScriptの型チェックとコンパイラの仕組み",
    "17.型安全な開発フロー"
]

chapter_titles_combined = "\n".join(chapter_titles)

# 出力ディレクトリの作成
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

async def main():
    pdf_files = []
    for title in tqdm(chapter_titles[15:], total=len(chapter_titles)):
        print(f"=== {title} ===")
        task_description = (
            f"{title}に関する章の内容を作成してください。\n"
            "注意:全体の構成は以下のとおりです。以下の構成で一つずつ該当の章の内容を作成していることを意識してください。\n"
            "なお、全体で100ページ程度の書籍にする必要があります。\n"
            f"{chapter_titles_combined}"
        )
        stream = agent_team.run_stream(task=task_description)
        chapter_content = ""
        async for message in stream:
            if isinstance(message, BaseMessage):
                print(f"{message.source}: {message.content}")
            else:
                result = message.messages[-2]
                chapter_content += f"{result.content}\n\n"
        
       # Markdownファイルとして保存
        md_filename = os.path.join(output_dir, f"{title}.md")
        with open(md_filename, "w", encoding="utf-8") as md_file:
            md_file.write(f"# {title}\n\n")
            md_file.write(chapter_content)


if __name__ == "__main__":
    asyncio.run(main())

  0%|          | 0/17 [00:00<?, ?it/s]

=== 01.TypeScriptの基礎 ===
user: 01.TypeScriptの基礎に関する章の内容を作成してください。
注意:全体の構成は以下のとおりです。以下の構成で一つずつ該当の章の内容を作成していることを意識してください。
なお、全体で100ページ程度の書籍にする必要があります。
01.TypeScriptの基礎
02.TypeScriptの型システム
03.インターフェースと型エイリアス
04.型推論とユーティリティ型
05.関数型プログラミングとTypeScript
06.ジェネリクスと高度な型
07.TypeScriptにおけるクラスとオブジェクト指向
08.TypeScriptとPythonの設計思想の違い
09.Pythonの型ヒントとTypeScriptの型の比較
10.TypeScriptでの非同期処理
11.型安全なAPI設計
12.TypeScriptでのCLIツール開発
13.フロントエンドとバックエンドの開発
14.型安全なデータ処理
15.TypeScriptの型システムの内部構造
16.TypeScriptの型チェックとコンパイラの仕組み
17.型安全な開発フロー
WriterAgent: # 01. TypeScriptの基礎

## 1.1 TypeScriptとは

TypeScriptは、JavaScriptのスーパーセットとして設計されたプログラミング言語です。Microsoftによって開発され、型安全性やクラスベースのオブジェクト指向プログラミングをサポートする機能を追加しています。TypeScriptは、JavaScriptのコードをそのまま利用できるため、既存のJavaScriptプロジェクトに簡単に取り入れることが可能です。TypeScriptで記述されたコードは、コンパイルを経てJavaScriptに変換され、ブラウザやNode.jsで実行できます。

PythonとTypeScriptは異なる言語でありながら、いくつかのデザインパターンやプログラミング思想が共通している部分もあります。特に、TypeScriptの型システムはPythonの型ヒントに類似しており、開発時の安全性を高めるために役立ちます。

## 1.2 TypeScriptの基本的な構文

TypeScriptの基本的な構文は、Java

Error processing publish message for ReviewerAgent/91c6a5b5-b862-4a58-a0a6-93c295e040b5
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 505, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/autogen_core/_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 48, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/autogen_core/_routed_agent.py", line 485, in on_message_impl
    return await h(self, message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-p

=== 17.型安全な開発フロー ===
user: 17.型安全な開発フローに関する章の内容を作成してください。
注意:全体の構成は以下のとおりです。以下の構成で一つずつ該当の章の内容を作成していることを意識してください。
なお、全体で100ページ程度の書籍にする必要があります。
01.TypeScriptの基礎
02.TypeScriptの型システム
03.インターフェースと型エイリアス
04.型推論とユーティリティ型
05.関数型プログラミングとTypeScript
06.ジェネリクスと高度な型
07.TypeScriptにおけるクラスとオブジェクト指向
08.TypeScriptとPythonの設計思想の違い
09.Pythonの型ヒントとTypeScriptの型の比較
10.TypeScriptでの非同期処理
11.型安全なAPI設計
12.TypeScriptでのCLIツール開発
13.フロントエンドとバックエンドの開発
14.型安全なデータ処理
15.TypeScriptの型システムの内部構造
16.TypeScriptの型チェックとコンパイラの仕組み
17.型安全な開発フロー


Error processing publish message for WriterAgent/91c6a5b5-b862-4a58-a0a6-93c295e040b5
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 505, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/autogen_core/_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 48, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/autogen_core/_routed_agent.py", line 485, in on_message_impl
    return await h(self, message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-pac

IndexError: list index out of range